In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import requests
import pandas as pd 
import xml.etree.ElementTree as ET
import re

In [3]:
# Request for elementary and middle schoools
city = input("What city? ")
state = input("What state? ")
distance = input("What distance? ")
limit = input("How many schools? ")
r1 = requests.get('https://api.greatschools.org/schools/nearby?key=8316df0fb319ba32b6a5e85f4413fd4a&city={}&state={}&schoolType=public&levelCode=elementary-schools&radius={}&limit={}'.format(city, state, distance, limit))
r2 = requests.get('https://api.greatschools.org/schools/nearby?key=8316df0fb319ba32b6a5e85f4413fd4a&city={}&state={}&schoolType=public&levelCode=middle-schools&radius={}&limit={}'.format(city, state, distance, limit))
r3 = requests.get('https://api.greatschools.org/schools/nearby?key=8316df0fb319ba32b6a5e85f4413fd4a&city={}&state={}&schoolType=public&levelCode=high-schools&radius={}&limit={}'.format(city, state, distance, limit))

What city? Columbia
What state? MD
What distance? 10
How many schools? 100


In [4]:
def FileMaker(html, text_to_write):
    file = open(text_to_write, "w")
    file.write(str(html))
    file.close() 

In [5]:
FileMaker(r1.text, "gs_ele.txt")
FileMaker(r2.text, "gs_mid.txt")
FileMaker(r3.text, "gs_high.txt")

In [6]:
tree = ET.parse("gs_ele.txt")
root_ele = tree.getroot()
tree = ET.parse("gs_mid.txt")
root_mid = tree.getroot()
tree = ET.parse("gs_high.txt")
root_high = tree.getroot()

ParseError: syntax error: line 1, column 0 (<string>)

In [ ]:
for school in root_ele.findall('school'):
    ename = school.find('name').text
    try:
        egsRating = school.find('gsRating').text
    except:
        egsRating = "No Rating"

    
for school in root_mid.findall('school'):
    mname = school.find('name').text
    try:
        mgsRating = school.find('gsRating').text
    except:
        mgsRating = "No Rating"
        
for school in root_high.findall('school'):
    hname = school.find('name').text
    try:
        hgsRating = school.find('gsRating').text
    except:
        hgsRating = "No Rating"

In [ ]:
eschool_list  = []
eid_list = []
erank_list = []
eenrollment_list = []

for school in root_ele.findall('school'):
    name = school.find('name').text
    eschool_list.append(name)
    try:
        gsID = school.find("gsId").text
    except:
        gsID = "*No ID*"
    eid_list.append(gsID)
    try:
        gsRating = school.find('gsRating').text
    except:
        gsRating = "0.1"
    erank_list.append( gsRating)
    
    try:
        enrollment = school.find("enrollment").text
    except:
        enrollment = "*No enrollment*"
    eenrollment_list.append(enrollment)
    

In [ ]:
mschool_list  = []
mid_list = []
mrank_list = []
menrollment_list = []
for school in root_mid.findall('school'):
    name = school.find('name').text
    mschool_list.append(name)
    
    try:
        gsID = school.find("gsId").text
    except:
        gsID = "***No ID***"
    mid_list.append(gsID)
    
    try:
        gsRating = float(school.find('gsRating').text)
    except:
        gsRating = "0.1"
    mrank_list.append( gsRating)
    
    try:
        enrollment = school.find("enrollment").text
    except:
        enrollment = "***No enrollment***"
    menrollment_list.append(enrollment)

In [ ]:
hschool_list  = []
hid_list = []
hrank_list = []
henrollment_list = []
for school in root_high.findall('school'):
    name = school.find('name').text
    hschool_list.append(name)
    
    try:
        gsID = school.find("gsId").text
    except:
        gsID = "***No ID***"
    hid_list.append(gsID)
    
    try:
        gsRating = float(school.find('gsRating').text)
    except:
        gsRating = "0.1"
    hrank_list.append( gsRating)
    
    try:
        enrollment = school.find("enrollment").text
    except:
        enrollment = "***No enrollment***"
    henrollment_list.append(enrollment)

In [ ]:
elunch = []
ewhite = []
eraces = []
easian = []
e_test = []    
e_progress = []
e_equity = []

for item in eid_list:
    r = requests.get('https://api.greatschools.org/school/census/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item))
    f = open("lunch.txt", "w")
    f.write(str(r.text))
    f.close()
    
    tree = ET.parse("lunch.txt")
    root = tree.getroot()
    
    try:   
        lunch = root.find('freeAndReducedPriceLunch').text
    except:
        lunch = "NaN"
    elunch.append(lunch)
    
    
    num = 0
    my_dict = {}
    thing = root.findall('.//name')
    for i in thing:
        my_dict.update( {num : i.text} )
        num +=1
    white = [key  for (key, value) in my_dict.items() if value == 'White, non-Hispanic']
    asian = [key  for (key, value) in my_dict.items() if value == 'Asian']
    
    
    
    eth = root.findall('.//value')
    try:
        ewhite.append(eth[white[0]].text)
    except:
        ewhite.append('NaN')
    try:
        easian.append(eth[asian[0]].text)
    except:
        easian.append('NaN')

    school_page = requests.get('https://api.greatschools.org/schools/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item)) 
    file = open("test_scores.txt", "w")
    file.write(str(school_page.text))
    file.close() 
    tree = ET.parse("test_scores.txt")
    root = tree.getroot()
    url = root.find("overviewLink")
    overview_page = requests.get(url.text)
    try:
        test_scores = re.findall('<span class="gs-rating circle-rating--(.*) circle-rating--xtra-small">', overview_page.text)
    except:
        test_scores = "NaN"
    try:
        e_test.append(test_scores[0])
    except:
        e_test.append("NaN")
    try:   
        e_progress.append(test_scores[1])
    except:
        e_progress.append("NaN")
    try:
        e_equity.append(test_scores[2])
    except:
        e_equity.append("NaN")

In [ ]:
mlunch = []
mwhite = []
mraces = []
masian = []
m_test = []    
m_progress = []
m_equity = []


for item in mid_list:
    r = requests.get('https://api.greatschools.org/school/census/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item))
    f = open("lunch.txt", "w")
    f.write(str(r.text))
    f.close()
    
    tree = ET.parse("lunch.txt")
    root = tree.getroot()
    
    try:   
        lunch = root.find('freeAndReducedPriceLunch').text
    except:
        lunch = "NaN"
    mlunch.append(lunch) 
    
    
    num = 0
    my_dict = {}
    thing = root.findall('.//name')
    for i in thing:
        my_dict.update( {num : i.text} )
        num +=1
    white = [key  for (key, value) in my_dict.items() if value == 'White, non-Hispanic']
    asian = [key  for (key, value) in my_dict.items() if value == 'Asian']
    
    
    
    eth = root.findall('.//value')
    try:
        mwhite.append(eth[white[0]].text)
    except:
        mwhite.append('NaN')
    try:
        masian.append(eth[asian[0]].text)
    except:
        masian.append('NaN')
        
     
    school_page = requests.get('https://api.greatschools.org/schools/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item)) 
    file = open("test_scores.txt", "w")
    file.write(str(school_page.text))
    file.close() 
    tree = ET.parse("test_scores.txt")
    root = tree.getroot()
    url = root.find("overviewLink")
    overview_page = requests.get(url.text)
    try:
        test_scores = re.findall('<span class="gs-rating circle-rating--(.*) circle-rating--xtra-small">', overview_page.text)
        
        
  
    except:
        test_scores = "NaN"
    try:
        m_test.append(test_scores[0])
    except:
        m_test.append("NaN")
    try:   
        m_progress.append(test_scores[1])
    except:
        m_progress.append("NaN")
    try:
        m_equity.append(test_scores[2])
    except:
        m_equity.append("NaN")   
            

In [ ]:
hlunch = []
hwhite = []
hraces = []
hasian = []
h_test = []    
h_progress = []
h_equity = []

for item in hid_list:
    r = requests.get('https://api.greatschools.org/school/census/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item))
    f = open("lunch.txt", "w")
    f.write(str(r.text))
    f.close()
    
    
    
    
    
    tree = ET.parse("lunch.txt")
    root = tree.getroot()
    
    try:   
        lunch = root.find('freeAndReducedPriceLunch').text
    except:
        lunch = "NaN"
    hlunch.append(lunch)
    
    hth = root.findall('.//value')
    
    num = 0
    my_dict = {}
    thing = root.findall('.//name')
    for i in thing:
        my_dict.update( {num : i.text} )
        num +=1
    white = [key  for (key, value) in my_dict.items() if value == 'White, non-Hispanic']
    asian = [key  for (key, value) in my_dict.items() if value == 'Asian']
    
    
    try:
        hwhite.append(hth[white[0]].text)
    except:
        hwhite.append('NaN')
    try:
        hasian.append(hth[asian[0]].text)
    except:
        hasian.append('NaN')

    school_page = requests.get('https://api.greatschools.org/schools/{}/{}?key=8316df0fb319ba32b6a5e85f4413fd4a'.format(state, item)) 
    file = open("test_scores.txt", "w")
    file.write(str(school_page.text))
    file.close() 
    tree = ET.parse("test_scores.txt")
    root = tree.getroot()
    url = root.find("overviewLink")
    overview_page = requests.get(url.text)
    try:
        test_scores = re.findall('<span class="gs-rating circle-rating--(.*) circle-rating--xtra-small">', overview_page.text)
        
        
  
    except:
        test_scores = "NaN"
    try:
        h_test.append(test_scores[0])
    except:
        h_test.append("NaN")
    try:   
        h_progress.append(test_scores[1])
    except:
        h_progress.append("NaN")
    try:
        h_equity.append(test_scores[2])
    except:
        h_equity.append("NaN")

In [ ]:
ele_d = {'School Name': eschool_list, 'Test Scores': e_test, 'Rating': erank_list,  'Lunch': elunch,  'White': ewhite, "Asian": easian, 'Enrollment': eenrollment_list, 'Progess': e_progress, 'Equity': e_equity, 'Id Number': eid_list}
ele_df = pd.DataFrame(ele_d)

In [ ]:
#ele_df['Lunch'] = ele_df['Lunch'].astype(float)
#ele_df.sort_values(by=['Lunch'])

# Elementary Schols

In [ ]:
ele_df['Test Scores'] = ele_df['Test Scores'].astype(float)
ele_df.sort_values(by=['Test Scores', 'White'], ascending=False)

In [ ]:
mid_d = {'School Name': mschool_list, 'Test Scores': m_test, 'Rating': mrank_list,  'Lunch': mlunch,  'White': mwhite, "Asian": masian, 'Enrollment': menrollment_list, 'Progess': m_progress, 'Equity': m_equity, 'Id Number': mid_list}
mid_df = pd.DataFrame(mid_d)



# Middle Schools

In [ ]:


mid_df['Test Scores'] = mid_df['Test Scores'].astype(float)
mid_df.sort_values(by=['Test Scores', 'White'], ascending=False)


# High Schools

In [ ]:
hid_d = {'School Name': hschool_list, 'Test Scores': h_test, 'Rating': hrank_list,  'Lunch': hlunch,  'White': hwhite, "Asian": hasian, 'Enrollment': henrollment_list, 'Progess': h_progress, 'Equity': h_equity, 'Id Number': hid_list}
hid_df = pd.DataFrame(hid_d)

In [ ]:
hid_df['Test Scores'] = hid_df['Test Scores'].astype(float)
hid_df.sort_values(by=['Test Scores', 'White'], ascending=False)

In [ ]:
writer = pd.ExcelWriter('{}_schools.xlsx'.format(city), engine='xlsxwriter')

# Write each dataframe to a different worksheet.
ele_df.to_excel(writer, sheet_name='{}_Elementary'.format(city))
mid_df.to_excel(writer, sheet_name='{}_Middle'.format(city))
hid_df.to_excel(writer, sheet_name='{}_High'.format(city))
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
hid_df.to_excel()

In [ ]:
>>> writer = pd.ExcelWriter('output.xlsx')
>>> df1.to_excel(writer,'Sheet1')
>>> df2.to_excel(writer,'Sheet2')